# Лабораторная работа 2. Метод ближайших соседей и решающие деревья.

ФИО: Измаилов Павел Алексеевич

Группа: 317

In [153]:
import numpy as np
import pandas as pd
import time
from sklearn import cross_validation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [154]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [155]:
data.shape

(32769, 10)

In [156]:
# доля положительных примеров
data.ACTION.mean()

0.94210992096188473

In [157]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

('ACTION', 2)
('RESOURCE', 7518)
('MGR_ID', 4243)
('ROLE_ROLLUP_1', 128)
('ROLE_ROLLUP_2', 177)
('ROLE_DEPTNAME', 449)
('ROLE_TITLE', 343)
('ROLE_FAMILY_DESC', 2358)
('ROLE_FAMILY', 67)
('ROLE_CODE', 343)


In [158]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

## Часть 1: kNN и категориальные признаки

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре. Реализуйте самостоятельно метод k ближайших соседей, который будет уметь работать с этими функциями расстояния (учитите, что он должен возвращать вероятность — отношение объектов первого класса среди соседей к числу соседей). Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba).

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

Какая функция расстояния оказалась лучшей?

#### Readme

В этом задании для ускорения работы knn я постарался максимально векторизовать вычисления. Для вычисления попарных расстояний я добавлял к данным третью ось, после чего вычислял попарные расстояния с использованием numpy broadcasting.

In [159]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Переведем все данные в numpy array

In [160]:
x_train_m = X_train.as_matrix()
x_test_m = X_test.as_matrix()
y_train_m = y_train.as_matrix()
y_test_m = y_test.as_matrix()

####Первая функция расстояния

Функция расстояния $\rho(x_i, y_i) = [x_i \ne y_i]$

In [161]:
def dist_1(x, y):
    return np.sum(x != y, axis=2).astype(float)

####Вторая функция расстояния

Отсортируем для каждой координаты признаки по частоте и построим список, состоящий из массивов — по массиву на каждый признак. В первом столбце массива находятся всевозможные значения $x$ данного признака $j$, а во втором — сумма $\sum\limits_{q: p_j(q) < p_j(x)} p_j^2(q)$, где $p_j^2(x) = f_j(x) (f_j(x) - 1) / l (l - 1)$, где $f_j(x)$ — количество примеров, в которых значение признака $j$ равно $x$ , а $l$ — число всевозможных значений признака. В третьем столбце находится $\log(f_j(x))$. При этом сверху находятся элементы с наименьшимим, а снизу — с наибольшими вероятностями

In [162]:
labels_sorted_by_freq = []
for column in X_train.columns:
    frequencies = np.array(((X_train[column].value_counts()).tolist()))[::-1]
    l = len(X_train[column])
    freq_diff = np.diff(frequencies)
    sqrd_prbs = (np.cumsum(frequencies * (frequencies-1)) / float(l * (l-1)))
    for i in xrange(len(freq_diff)):
        if (freq_diff[i] == 0):            
            sqrd_prbs[i] = np.max(sqrd_prbs[frequencies == frequencies[i]])
    sqrd_prbs = sqrd_prbs.tolist()
    freq_logs = (np.log(frequencies + np.ones(frequencies.shape))).tolist()
    labels_sorted_by_freq.append(np.array(
        [((X_train[column].value_counts()).index.tolist())[::-1], sqrd_prbs, freq_logs]).T)

In [163]:
labels_sorted_by_freq[0]

array([[  7.99990000e+04,   0.00000000e+00,   6.93147181e-01],
       [  7.98050000e+04,   0.00000000e+00,   6.93147181e-01],
       [  2.85980000e+04,   0.00000000e+00,   6.93147181e-01],
       ..., 
       [  7.50780000e+04,   1.68575037e-03,   5.70044357e+00],
       [  7.90920000e+04,   1.89336109e-03,   5.80513497e+00],
       [  4.67500000e+03,   2.54715904e-03,   6.37672695e+00]])

In [164]:
a = np.array([1, 2, 3, 4, 5, 5, 5, 6])
np.diff(a)

array([1, 1, 1, 1, 0, 0, 1])

Функция corrections получает на вход вектор-столбец из значений признака column в точках x, и возвращает массив значений сумм квадратов вероятностей значений этого признака, меньших, чем значения этого признака у x. С ее Помощью в dist_2 вычисляется метрика $\rho(x_i, y_i) = [x_i \ne y_i] + [x_i = y_i] \sum\limits_{q: p_i(q) \le p_i(x_i)} p_i^2(q)$.

In [165]:
def corrections(x, column):
    correction_matrix = labels_sorted_by_freq[column]
    label_values_stacked = np.vstack([correction_matrix[:, 0]]*x.shape[0])
    probs_stacked = np.vstack([correction_matrix[:, 1]]*x.shape[0])
    mat = label_values_stacked == x
    mat = np.hstack([mat, np.logical_not(np.sum(mat, axis=1))[:, None]])
    probs_stacked = np.hstack([probs_stacked, np.zeros((probs_stacked.shape[0], 1))])
    return (probs_stacked)[mat].reshape(x.shape)

def dist_2(x, y):
    dist = dist_1(x, y)
    for i in range(x.shape[2]):
        correction_matrix = np.hstack([corrections(x[:, :, i], i)] * dist.shape[1])
        correction_matrix[(x[:, :, i] != y[:, :, i])] = 0
        dist += correction_matrix

    return dist
    

###### Третья функция расстояния
$\rho_j(x_j, y_j) = [x_j \ne y_j] \log(f_j(x_j)) \log(f_j(y_j))$

In [166]:
def log_corrections(x, column):
    correction_matrix = labels_sorted_by_freq[column]
    label_values_stacked = np.vstack([correction_matrix[:, 0]]*x.shape[0])
    log_freqs_stacked = np.vstack([correction_matrix[:, 2]]*x.shape[0])
    mat = label_values_stacked == x
    mat = np.hstack([mat, np.logical_not(np.sum(mat, axis=1))[:, None]])
    log_freqs_stacked = np.hstack([log_freqs_stacked, np.zeros((log_freqs_stacked.shape[0], 1))])
    return (log_freqs_stacked)[mat].reshape(x.shape)

def dist_3(x, y):
    dist = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[2]):
        correction_matrix = log_corrections(y[:, :, i].T, i).T * log_corrections(x[:, :, i], i)
        correction_matrix[(x[:, :, i] == y[:, :, i])] = 1
        dist += correction_matrix*(x[:, :, i] != y[:, :, i])
    return dist
    

In [167]:
class  k_neighbour_classifier:
    def __init__(self, distance, num_neigbours):
        self.dist = distance
        self.k = num_neigbours
        
    def predict(self, train_x, train_y, test_x):
        """Returns predicted labels at test data points"""
        distance_matrix = self.dist(test_x[:, None, :], train_x[None, :, :])
        args = np.argsort(distance_matrix, axis=1)[:, :self.k]
        args = args.flatten()
        y_indices_list = np.hstack([np.arange(test_x.shape[0])[:, None]] * self.k).ravel()
        labels_matrix = np.vstack([train_y]*test_x.shape[0])
        sorted_dist_labels_matrix = np.array(np.hsplit(labels_matrix[y_indices_list, args.flatten()], test_x.shape[0]))
        return np.sum(sorted_dist_labels_matrix, axis=1)/float(self.k)

In [168]:
knn = k_neighbour_classifier(dist_1, 10)
predicted_y = knn.predict(x_train_m, y_train_m, x_test_m)
print 'Метрика roc-auc для первой функции расстояния: ', roc_auc_score(y_test_m, predicted_y)

Метрика roc-auc для первой функции расстояния:  0.83088009598


In [169]:
knn = k_neighbour_classifier(dist_2, 10)
predicted_y = knn.predict(x_train_m, y_train_m, x_test_m)
print 'Метрика roc-auc для второй функции расстояния: ', roc_auc_score(y_test_m, predicted_y)

Метрика roc-auc для второй функции расстояния:  0.832873632128


In [170]:
knn = k_neighbour_classifier(dist_3, 10)
predicted_y = knn.predict(x_train_m, y_train_m, x_test_m)
print 'Метрика roc-auc для третьей функции расстояния: ', roc_auc_score(y_test_m, predicted_y)

Метрика roc-auc для третьей функции расстояния:  0.820966777872


###Ответ на 1.1 
Лучшей оказалась вторая функция расстояния

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось достичь?

####Первая функция расстояния

In [171]:
k_vals = [1, 3, 5, 10, 15, 50, 100]
max_quality = 0
k_opt = 0
for k in k_vals:
    print(k)
    knn = k_neighbour_classifier(dist_1, k)
    predicted_y = knn.predict(x_train_m, y_train_m, x_test_m)
    quality = roc_auc_score(y_test_m, predicted_y)
    if max_quality < quality:
        max_quality = quality
        k_opt = k
print("Наилучшее качество ", max_quality, " достигается при числе соседей k = ", k_opt)

1
3
5
10
15
50
100
('\xd0\x9d\xd0\xb0\xd0\xb8\xd0\xbb\xd1\x83\xd1\x87\xd1\x88\xd0\xb5\xd0\xb5 \xd0\xba\xd0\xb0\xd1\x87\xd0\xb5\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe ', 0.83088009598014845, ' \xd0\xb4\xd0\xbe\xd1\x81\xd1\x82\xd0\xb8\xd0\xb3\xd0\xb0\xd0\xb5\xd1\x82\xd1\x81\xd1\x8f \xd0\xbf\xd1\x80\xd0\xb8 \xd1\x87\xd0\xb8\xd1\x81\xd0\xbb\xd0\xb5 \xd1\x81\xd0\xbe\xd1\x81\xd0\xb5\xd0\xb4\xd0\xb5\xd0\xb9 k = ', 10)


In [172]:
k_vals = [1, 3, 5, 10, 15, 50, 100]
max_quality = 0
k_opt = 0
for k in k_vals:
    print(k)
    knn = k_neighbour_classifier(dist_2, k)
    predicted_y = knn.predict(x_train_m, y_train_m, x_test_m)
    quality = roc_auc_score(y_test_m, predicted_y)
    if max_quality < quality:
        max_quality = quality
        k_opt = k
print("Наилучшее качество ", max_quality, " достигается при числе соседей k = ", k_opt)

1
3
5
10
15
50
100
('\xd0\x9d\xd0\xb0\xd0\xb8\xd0\xbb\xd1\x83\xd1\x87\xd1\x88\xd0\xb5\xd0\xb5 \xd0\xba\xd0\xb0\xd1\x87\xd0\xb5\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe ', 0.83287363212833387, ' \xd0\xb4\xd0\xbe\xd1\x81\xd1\x82\xd0\xb8\xd0\xb3\xd0\xb0\xd0\xb5\xd1\x82\xd1\x81\xd1\x8f \xd0\xbf\xd1\x80\xd0\xb8 \xd1\x87\xd0\xb8\xd1\x81\xd0\xbb\xd0\xb5 \xd1\x81\xd0\xbe\xd1\x81\xd0\xb5\xd0\xb4\xd0\xb5\xd0\xb9 k = ', 10)


In [173]:
k_vals = [1, 3, 5, 10, 15, 50, 100]
max_quality = 0
k_opt = 0
for k in k_vals:
    print(k)
    knn = k_neighbour_classifier(dist_3, k)
    predicted_y = knn.predict(x_train_m, y_train_m, x_test_m)
    quality = roc_auc_score(y_test_m, predicted_y)
    if max_quality < quality:
        max_quality = quality
        k_opt = k
print "Наилучшее качество ", max_quality, " достигается при числе соседей k = ", k_opt

1
3
5
10
15
50
100
('\xd0\x9d\xd0\xb0\xd0\xb8\xd0\xbb\xd1\x83\xd1\x87\xd1\x88\xd0\xb5\xd0\xb5 \xd0\xba\xd0\xb0\xd1\x87\xd0\xb5\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe ', 0.82096677787194638, ' \xd0\xb4\xd0\xbe\xd1\x81\xd1\x82\xd0\xb8\xd0\xb3\xd0\xb0\xd0\xb5\xd1\x82\xd1\x81\xd1\x8f \xd0\xbf\xd1\x80\xd0\xb8 \xd1\x87\xd0\xb8\xd1\x81\xd0\xbb\xd0\xb5 \xd1\x81\xd0\xbe\xd1\x81\xd0\xb5\xd0\xb4\xd0\xb5\xd0\xb9 k = ', 10)


#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `clicks` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`clicks` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `clicks` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

####Без фолдинга

Выделим необходимые признаки

In [174]:
def calculate_counts(data_frame):
    new_data = pd.DataFrame()
    x_tr, x_test, y_tr, y_test = train_test_split(data_frame.iloc[:, 1:], data_frame.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
    new_data['ACTION'] = data_frame['ACTION']
    for col in data_frame.columns[1:]:
        new_data[col+'_COUNTS'] = (x_tr[col].value_counts())[data_frame[col]].tolist()
        new_data[col+'_CLICKS'] = ((x_tr[y_tr == 1])[col].value_counts())[data_frame[col]].tolist()
        new_data = new_data.fillna(0)
        new_data[col+'_FRACTION'] = (new_data[col+'_CLICKS'] + 1) / (new_data[col+'_COUNTS'] + 2)
    new_data = new_data.fillna(0)
    return new_data

In [175]:
def normalize_frame(df):
    df = (df - df.mean()) / (df.max() - df.min())
    return df

In [176]:
new_data = calculate_counts(data)
counts_X_train, counts_X_test, counts_y_train, counts_y_test = \
            train_test_split(new_data.iloc[:, 1:], new_data.iloc[:, 0], test_size=0.3, random_state=241)
counts_X_train = normalize_frame(counts_X_train)
counts_X_test = normalize_frame(counts_X_test)

In [177]:
k_vals = [1, 3, 5, 10, 15, 50, 100]
max_quality = 0
k_opt = 0
for k in k_vals:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(counts_X_train, counts_y_train)
    count_predicted_y = clf.predict_proba(counts_X_test)
    quality = roc_auc_score(counts_y_test.astype(float), count_predicted_y[:, 1])
    if max_quality < quality:
        max_quality = quality
        k_opt = k
print "Наилучшее качество ", max_quality, " достигается при числе соседей k = ", k_opt

Наилучшее качество  0.794847195176  достигается при числе соседей k =  50


#### С фолдингом

In [195]:
def calculate_folded_counts(data_frame, num_folds):
    x_tr, x_test, y_tr, y_test = train_test_split(data_frame.iloc[:, 1:], data_frame.iloc[:, 0],
                                                    test_size=0.3, random_state=241)
    kf = cross_validation.KFold(x_tr.shape[0], n_folds=num_folds)
    new_data = pd.DataFrame()
    new_data['ACTION'] = data['ACTION']
    for col in data_frame.columns[1:]:
        new_data[col+'_COUNTS'] = [0]*len(data_frame[col])
        new_data[col+'_CLICKS'] = [0]*len(data_frame[col])
        new_data[col+'_FRACTION'] = [0]*len(data_frame[col])
    for train_index, test_index in kf:
        x_in, x_left_out = x_tr.iloc[train_index], x_tr.iloc[test_index]
        y_in, y_left_out = y_tr.iloc[train_index], y_tr.iloc[test_index]
        for col in data.columns[1:]:
            new_data.loc[x_left_out.index, [col+'_COUNTS']] = (x_in[col].value_counts())[x_left_out[col]].tolist()
            new_data.loc[x_left_out.index, [col+'_CLICKS']] = \
                            ((x_in[y_in == 1])[col].value_counts())[x_left_out[col]].tolist()
            new_data.loc[x_left_out.index, [col+'_COUNTS']] /= float(y_in.size)
            new_data.loc[x_left_out.index, [col+'_CLICKS']] /= float(y_in.size)
            new_data = new_data.fillna(0)
    for col in data_frame.columns[1:]:
        new_data.loc[x_test.index, [col+'_COUNTS']] = (x_tr[col].value_counts())[x_test[col]].tolist()
        new_data.loc[x_test.index, [col+'_CLICKS']] = ((x_tr[y_tr==1])[col].value_counts())[x_test[col]].tolist()
        new_data.loc[x_test.index, [col+'_COUNTS']] /= float(y_tr.size)
        new_data.loc[x_test.index, [col+'_CLICKS']] /= float(y_tr.size)
        new_data = new_data.fillna(0)
    for col in data_frame.columns[1:]:
        new_data[col+'_FRACTION'] = ((new_data[col+'_CLICKS'] + 1.) / 
                                                            (new_data[col+'_COUNTS'] + 2.))
    return new_data

In [179]:
new_data = calculate_folded_counts(data, 3)
fold_x_train, fold_x_test, fold_y_train, fold_y_test = \
                    train_test_split(new_data.iloc[:, 1:], data.iloc[:, 0], test_size=0.3, random_state=241)
fold_x_train = normalize_frame(fold_x_train)
fold_x_test = normalize_frame(fold_x_test)

In [180]:
k_vals = [1, 3, 5, 10, 15, 50, 100]
max_quality = 0
k_opt = 0
for k in k_vals:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(fold_x_train, fold_y_train)
    fold_predicted_y = clf.predict_proba(fold_x_test)
    quality = roc_auc_score(fold_y_test.astype(float), fold_predicted_y[:, 1])
    if max_quality < quality:
        max_quality = quality
        k_opt = k
print "Наилучшее качество ", max_quality, " достигается при числе соседей k = ", k_opt

Наилучшее качество  0.776369291384  достигается при числе соседей k =  15


#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $f_i$, $f_j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$. Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

In [198]:
paired_data = data.copy()
for i in range(len(data.columns[1:])):
    for j in range(i+1, len(data.columns[1:])):
        column1 = data.columns[i+1]
        column2 = data.columns[j+1]
        paired_data[column1 + '_+_' + column2] = list(zip(data[column1], data[column2]))
            

####Без Фолдинга

In [200]:
paired_data_counts = calculate_counts(paired_data)

In [183]:
pc_x_tr, pc_x_test, pc_y_tr, pc_y_test = train_test_split(paired_data_counts.iloc[:, 1:], 
                                                paired_data_counts.iloc[:, 0], test_size=0.3, random_state=241)
pc_x_tr = normalize_frame(pc_x_tr)
pc_x_test = normalize_frame(pc_x_test)

In [190]:
k_vals = [1, 3, 5, 10, 15, 50, 100]
max_quality = 0
k_opt = 0
for k in k_vals:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(pc_x_tr, pc_y_tr)
    pc_predicted_y = clf.predict_proba(pc_x_test)
    quality = roc_auc_score(pc_y_test.astype(float), pc_predicted_y[:, 1])
    if max_quality < quality:
        max_quality = quality
        k_opt = k
print "Наилучшее качество ", max_quality, " достигается при числе соседей k = ", k_opt

Наилучшее качество  0.805653228583  достигается при числе соседей k =  100


####С Фолдингом

In [202]:
folded_paired_data_counts = calculate_folded_counts(paired_data, 3)

In [211]:
pfc_x_tr, pfc_x_test, pfc_y_tr, pfc_y_test = train_test_split(folded_paired_data_counts.iloc[:, 1:], 
                                            folded_paired_data_counts.iloc[:, 0], test_size=0.3, random_state=241)
pfc_x_tr = normalize_frame(pfc_x_tr).fillna(0)
pfc_x_test = normalize_frame(pfc_x_test)

In [213]:
k_vals = [1, 3, 5, 10, 15, 50, 100]
max_quality = 0
k_opt = 0
for k in k_vals:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(pfc_x_tr, pfc_y_tr)
    pfc_predicted_y = clf.predict_proba(pfc_x_test)
    quality = roc_auc_score(pfc_y_test.astype(float), pfc_predicted_y[:, 1])
    if max_quality < quality:
        max_quality = quality
        k_opt = k
print "Наилучшее качество ", max_quality, " достигается при числе соседей k = ", k_opt

Наилучшее качество  0.776369291384  достигается при числе соседей k =  15


## Часть 2: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

In [214]:
from sklearn.tree import DecisionTreeClassifier

In [216]:
depth_vals = [3, 5, 10, 20, 100]
min_samples_vals = [1, 2, 5, 10, 100, 1000]
max_quality = 0
opt_depth = 0
opt_min_sample = 0
for depth in depth_vals:
    for min_sample in min_samples_vals:        
        clf = DecisionTreeClassifier(random_state=0, max_depth=depth, min_samples_leaf=min_sample)
        clf.fit(pc_x_tr, pc_y_tr)
        tree_pc_prediction = clf.predict_proba(pc_x_test)
        quality = roc_auc_score(pc_y_test.astype(float), tree_pc_prediction[:, 1])
        if max_quality < quality:
            max_quality = quality
            opt_depth = depth
            opt_min_sample = min_sample
print "Наилучшее качество ", max_quality, " достигается при глубине ", opt_depth, \
      " и min_samples_leaf = ", opt_min_sample 

Наилучшее качество  0.753636006501  достигается при глубине  10  и min_samples_leaf =  5


#### 2. Настройте случайный лес, подобрав оптимальное число деревьев `n_estimators`. Какое качество на тестовой выборке он дает?

In [217]:
from sklearn.ensemble import RandomForestClassifier

In [218]:
n_vals = [10, 20, 50, 100, 200, 500]
max_quality = 0
opt_n = 0
for n in n_vals:
    clf = RandomForestClassifier(n_estimators=n)
    clf.fit(pc_x_tr, pc_y_tr)
    forest_pc_prediction = clf.predict_proba(pc_x_test)
    quality = roc_auc_score(pc_y_test.astype(float), forest_pc_prediction[:, 1])
    if max_quality < quality:
            max_quality = quality
            opt_n = n
print "Наилучшее качество ", max_quality, " достигается при числе деревьев ", opt_n 

Наилучшее качество  0.804530393115  достигается при числе деревьев  200


#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [219]:
n_vals = [10, 20, 50, 100, 200, 500]
max_quality = 0
opt_n = 0
for n in n_vals:
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(pfc_x_tr, pfc_y_tr)
    forest_pfc_prediction = clf.predict_proba(pfc_x_test)
    quality = roc_auc_score(pfc_y_test.astype(float), forest_pfc_prediction[:, 1])
    if max_quality < quality:
        max_quality = quality
        opt_n = n
print "Наилучшее качество ", max_quality, " достигается при числе деревьев ", opt_n

Наилучшее качество  0.812802408992  достигается при числе деревьев  50


Одной из причин этого эффекта является то, что в обучающей выборке многие признаки (а особенно парные) принимают какие-то из значений ровно по одному разу. Решающее дерево легко может переобучиться на таких признаках. При использовании фолдинга такие признаки не принимаются во внимание (для них counts и clicks равны 0). Да и вообще фолдинг понижает вероятность переобучения, так как при этой стратегии подсчета счетчиков мы не вносим в выборку информацию о целевой переменной.